In [1]:
from mxnet import ndarray as nd
import mxnet as mx
from mxnet.gluon import nn
from mxnet import gluon

In [2]:
net = nn.Sequential()
net.add(nn.Dense(128,activation='relu'))
net.add(nn.Dense(64,activation='relu'))
net.add(nn.Dense(10))
input_shape = (1,28,28)


In [4]:
ctx = [mx.gpu()]
net.initialize(mx.init.Xavier(magnitude=2.24),ctx=ctx)
# Mxnet特性：神经网络在第一次前向传播时初始化
# 因此初始化神经网络时需要做一次随机前向传播
net(nd.random.uniform(shape=input_shape,ctx=ctx[0]))
    


[[ 0.5636209   0.17707197 -0.09491384  0.28692648  0.07168481 -0.06108518
   0.38949418  0.16036497  0.37327743 -0.04009286]]
<NDArray 1x10 @gpu(0)>

In [5]:
net[0]

Dense(784 -> 128, Activation(relu))

In [9]:
test = net[0].weight.data()[:]

In [10]:
test


[[ 0.00684245  0.01301453  0.03016423 ...  0.04115127 -0.0447928
  -0.02196476]
 [ 0.05234559 -0.0685297   0.06567784 ... -0.03396226  0.01955298
  -0.04929858]
 [-0.03878776 -0.02429853  0.00324693 ... -0.02933703 -0.00237525
  -0.00157413]
 ...
 [ 0.02152947 -0.04127859 -0.05396653 ... -0.04758476  0.00495832
  -0.03560241]
 [ 0.01598034  0.05755057  0.06023888 ... -0.01171412 -0.01080731
   0.0321511 ]
 [ 0.04746842  0.01717804  0.02904288 ...  0.02043695 -0.04642513
  -0.02782338]]
<NDArray 128x784 @gpu(0)>

In [11]:
net[1].weight.data()


[[-0.04163705 -0.05344895  0.02144545 ... -0.11416268  0.11710194
   0.05810845]
 [ 0.13862053  0.09110172  0.01350582 ...  0.05885573 -0.1132526
   0.07703005]
 [ 0.038167   -0.1382699   0.12082759 ...  0.13215986  0.13550374
  -0.1267525 ]
 ...
 [ 0.14809904  0.04074837  0.00947204 ... -0.11959229 -0.05536778
   0.00356057]
 [ 0.05001102 -0.02192807 -0.09722202 ... -0.06633429 -0.00254634
   0.05555394]
 [ 0.00310361 -0.12117757  0.04927482 ... -0.09655102  0.05398586
   0.13996577]]
<NDArray 64x128 @gpu(0)>

In [13]:
weight = []
weight.append(net[0].weight.data()[:])

In [14]:
weight[0]


[[ 0.00684245  0.01301453  0.03016423 ...  0.04115127 -0.0447928
  -0.02196476]
 [ 0.05234559 -0.0685297   0.06567784 ... -0.03396226  0.01955298
  -0.04929858]
 [-0.03878776 -0.02429853  0.00324693 ... -0.02933703 -0.00237525
  -0.00157413]
 ...
 [ 0.02152947 -0.04127859 -0.05396653 ... -0.04758476  0.00495832
  -0.03560241]
 [ 0.01598034  0.05755057  0.06023888 ... -0.01171412 -0.01080731
   0.0321511 ]
 [ 0.04746842  0.01717804  0.02904288 ...  0.02043695 -0.04642513
  -0.02782338]]
<NDArray 128x784 @gpu(0)>